In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count, max, date_format,to_timestamp



# loading to df
customers_df = spark.read.table("Silver_layer.customers")
branches_df = spark.read.table("Silver_layer.branches")

transactions_df = spark.readStream.table("Silver_layer.transactions") 
customer_segment_df = spark.readStream.table("Silver_layer.customer_segments")
fraud_flags_df = spark.readStream.table("Silver_layer.fraud_flags") 

# 1. total transaction amount by branch
total_transaction_amount_by_branch = transactions_df.groupBy("branch_id") \
    .agg(sum("amount").alias("total_amount")) \
    .orderBy("branch_id")



# 3. number of transactions by channel and status
transaction_count_by_branch_and_channel = transactions_df.groupBy("channel", "status") \
    .agg(count("*").alias("transaction_count")) \
    .orderBy("channel", "status")

# 4. highest transaction amount by branch and customer
highest_transaction_by_branch_customer = transactions_df.groupBy("branch_id", "customer_id") \
    .agg(max("amount").alias("highest_amount")) \
    .orderBy("branch_id", "customer_id")

# 5. monthly transaction count and total Amount
monthly_transactions = transactions_df.withColumn("month", date_format(col("timestamp"), "yyyy-MM")) \
    .groupBy("month") \
    .agg(count("*").alias("transaction_count"), sum("amount").alias("total_amount")) \
    .orderBy("month")

# additional aggregations
# 6. transaction volumes by channel
transaction_volumes = transactions_df.groupBy("channel") \
    .agg(count("*").alias("total_transactions")) \
    .orderBy("channel")

# 7. transaction geographic distribution by branch location
transaction_geo_dist = transactions_df.join(branches_df, "branch_id") \
    .groupBy("location") \
    .agg(count("*").alias("transaction_count")) \
    .orderBy("location")


# 10. transaction patterns by month
transaction_patterns = transactions_df.withColumn("month", date_format(col("timestamp"), "yyyy-MM")) \
    .groupBy("month", "transaction_type") \
    .agg(count("*").alias("transaction_count"), sum("amount").alias("total_amount")) \
    .orderBy("month", "transaction_type")

# # Show the results
# total_transaction_amount_by_branch.show()
# average_transaction_amount_by_customer.show()
# transaction_count_by_branch_and_channel.show()
# highest_transaction_by_branch_customer.show()
# monthly_transactions.show()
# transaction_volumes.show()
# transaction_geo_dist.show()

# transaction_patterns.show()

# Save the results to Delta tables

total_transaction_amount_by_branch.writeStream.format("delta").option("checkpointLocation", "Gold_layer.checkpoint.tot_trans").outputMode("complete").toTable("Gold_layer.Tot_trans_amt_by_branch")
transaction_count_by_branch_and_channel.writeStream.format("delta").option("checkpointLocation", "Gold_layer.checkpoint.trans_count").outputMode("complete").toTable("Gold_layer.trans_count_by_branch_and_chnl")
highest_transaction_by_branch_customer.writeStream.format("delta").option("checkpointLocation", "Gold_layer.checkpoint.trans_high").outputMode("complete").toTable("Gold_layer.high_trans_by_branch_cus")
monthly_transactions.writeStream.format("delta").option("checkpointLocation", "Gold_layer.checkpoint.trans_month").outputMode("complete").toTable("Gold_layer.monthly_trans")
transaction_volumes.writeStream.format("delta").option("checkpointLocation", "Gold_layer.checkpoint.trans_vol").outputMode("complete").toTable("Gold_layer.trans_volumes")
transaction_geo_dist.writeStream.format("delta").option("checkpointLocation", "Gold_layer.checkpoint.trans_geo").outputMode("complete").toTable("Gold_layer.trans_geo_distinct")
transaction_patterns.writeStream.format("delta").option("checkpointLocation", "Gold_layer.checkpoint.trans_pattern").outputMode("complete").toTable("Gold_layer.trans_patterns")


java.io.IOException: Connection failed
	at com.databricks.rpc.Jetty9Client$$anon$1.handleError(Jetty9Client.scala:848)
	at com.databricks.rpc.Jetty9Client$$anon$1.onFailure(Jetty9Client.scala:775)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:197)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:189)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.notifyFailureComplete(HttpExchange.java:275)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.abort(HttpExchange.java:247)
	at shaded.v9_4.org.eclipse.jetty.client.HttpConversation.abort(HttpConversation.java:164)
	at shaded.v9_4.org.eclipse.jetty.client.HttpRequest.abort(HttpRequest.java:821)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.abort(HttpDestination.java:559)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.failed(HttpDestination.java:313)
	at com.databricks.rpc.AbstractConnectionPool$1.failed(AbstractConnectionPool.java:161)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at com.databricks.rpc.Jetty9Client$DatabricksHttpDestinationOverHTTP$$anon$2.failed(Jetty9Client.scala:1671)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at shaded.v9_4.org.eclipse.jetty.client.HttpClient$1$1.failed(HttpClient.java:660)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport.connectFailed(AbstractConnectorHttpClientTransport.java:138)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport$ClientSelectorManager.connectionFailed(AbstractConnectorHttpClientTransport.java:188)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$Connect.failed(ManagedSelector.java:966)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:369)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1700(ManagedSelector.java:65)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:676)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:535)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:362)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:186)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:173)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:131)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:409)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$4(InstrumentedQueuedThreadPool.scala:173)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:279)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:275)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:46)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:43)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool.withAttributionContext(InstrumentedQueuedThreadPool.scala:125)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$3(InstrumentedQueuedThreadPool.scala:173)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads(QueuedThreadPoolInstrumenter.scala:110)
	at com.databricks.instrumentation.QueuedThreadPoolInstrume